# Imports and helper functions

In [1]:
import pandas as pd
from swifter import swifter
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import TransformerMixin
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.inspection import permutation_importance
from sklearn import model_selection, metrics
import sagemaker
import awscli
import os
import s3fs



AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
s3 = s3fs.S3FileSystem(
    anon=False, key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY)


def data_description(df):
    print('Variables:\n\n{}'.format(df.dtypes), end='\n\n')
    print('Number of rows {}'.format(df.shape[0]), end='\n\n')
    print('Number of columns {}'.format(df.shape[1]), end='\n\n')
    print('NA analysis'.format(end='\n'))
    for i in df.columns:
        print('column {}: {} {}'.format(i,df[i].isna().any(), df[i].isna().sum()))

def consult_table_athena(database, table):
    wr.config.aws_profile = 'default'
    wr.config.region = 'us-east-1'

    query = f"SELECT * FROM {database}.{table}"

    df = wr.athena.read_sql_query(query, database=database)

    return df


def unique_values_columns(df):
    """
    Display unique values for each object (or string) column in a DataFrame.
    
    Parameters:
    - df (DataFrame): Input DataFrame
    
    Returns:
    - dict: A dictionary with column names as keys and unique values as lists.
    """
    
    # Filter out only object or string type columns
    object_cols = df.select_dtypes(include=['object']).columns
    
    # Get unique values for each object column
    unique_values = {col: df[col].unique().tolist() for col in object_cols}
    
    return unique_values

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/dbcordeiro@sefaz.al.gov.br/.config/sagemaker/config.yaml


# Loading Data

In [15]:
# Loading raw data from athena
# Balancing classes
# Separate train and test data


# DATABASE = 'respiratory_db'
# TABLE = 'table_respiratory_traintrain_data'
# df = consult_table_athena(DATABASE, TABLE)

# df.to_parquet('data.parquet', index=False)

# df_raw['classi_fin'].value_counts().sort_index(ascending=True)
# class1 = df_raw[df_raw['classi_fin'] == 1].sample(31437, random_state=42)
# class2 = df_raw[df_raw['classi_fin'] == 2]
# class3 = df_raw[df_raw['classi_fin'] == 3]
# class4 = df_raw[df_raw['classi_fin'] == 4].sample(31437, random_state=42)
# class5 = df_raw[df_raw['classi_fin'] == 5].sample(31437, random_state=42)
# df_raw = pd.concat([class1, class2, class3, class4, class5], ignore_index=True)

# train, test = train_test_split(df_raw, test_size=0.15, random_state=42)

# train.to_parquet('train.parquet', index=False)
# test.to_parquet('test.parquet', index=False)

NameError: name 'df_raw' is not defined

In [5]:
df = pd.read_parquet('../data/train.parquet')

## Data description

In [6]:
df.columns = df.columns.str.lower()

In [114]:
data_description(df)

Variables:

sem_not                Int64
sem_pri                Int64
sg_uf_not     string[python]
id_regiona    string[python]
co_regiona           float64
id_municip    string[python]
co_mun_not             Int64
cs_sexo       string[python]
nu_idade_n             Int64
tp_idade               Int64
cod_idade              Int64
cs_gestant             Int64
cs_raca                Int64
cs_escol_n           float64
sg_uf         string[python]
cs_zona              float64
surto_sg             float64
nosocomial           float64
ave_suino            float64
febre                float64
tosse                float64
garganta             float64
dispneia             float64
desc_resp            float64
saturacao            float64
diarreia             float64
vomito               float64
outro_sin            float64
outro_des     string[python]
puerpera             float64
fator_risc             Int64
cardiopati           float64
hematologi           float64
sind_down            float64
he

In [7]:
df = reduce_mem_usage(df)

Mem. usage decreased to 36.83 Mb (43.3% reduction)


In [56]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sem_not,128700.0,22.604452,14.454449,1.0,11.0,21.0,33.0,52.0
sem_pri,128700.0,22.483504,14.83256,1.0,10.0,20.0,33.0,52.0
co_regiona,114809.0,NaN,NaN,1331.0,1345.0,1385.0,1519.0,6256.0
co_mun_not,128700.0,353919.485703,86520.467632,110002.0,311860.0,354140.0,411370.0,530010.0
nu_idade_n,128700.0,42.614087,30.5641,-9.0,7.0,48.0,69.0,123.0
tp_idade,128700.0,2.838003,0.39574,1.0,3.0,3.0,3.0,3.0
cod_idade,128699.0,2880.624107,411.838062,1000.0,3003.0,3048.0,3069.0,3123.0
cs_gestant,128700.0,5.819246,0.800761,0.0,5.0,6.0,6.0,9.0
cs_raca,128700.0,3.53101,2.888806,1.0,1.0,4.0,4.0,9.0
cs_escol_n,84356.0,NaN,0.0,0.0,2.0,5.0,9.0,9.0


### Data Cleaning / NA analysis / Outliers analysis

In [8]:
# Sexo have Male, F and I, lets take off the I value has there are just 188 lines.
df = df.loc[df['cs_sexo'] != 'I']
df['cs_sexo'] = df['cs_sexo'].astype('category')

df = df.drop('delta_uti', axis=1)

# Negative ages are excluded
df = df[~df['nu_idade_n'] <= 0]

# Remove demographic categories

df = df.drop(
    [
    'sg_uf_not',
    'id_regiona',
    'co_regiona',
    'id_municip',
    'co_mun_not',
    'sg_uf',
    'cod_idade',
    'cs_escol_n'
    ], axis=1
)

df['tp_amostra'].replace(df['tp_amostra'].max(),df['tp_amostra'].mode()[0], inplace=True)
df['dor_abd'].replace(df['dor_abd'].max(),df['dor_abd'].mode()[0], inplace=True)
df['perd_olft'].replace(df['perd_olft'].max(),df['perd_olft'].mode()[0], inplace=True)

In [9]:
float_cols = df.select_dtypes(['float16','float32','int64']).columns
df[float_cols] = df[float_cols].swifter.apply(
    lambda x: x.fillna(x.mode()[0])
)

df[float_cols] = df[float_cols].astype('int8')

int_cols = df.select_dtypes(['int64','int8']).drop(['sem_not','sem_pri','nu_idade_n'], axis=1).columns
df[int_cols] = df[int_cols].astype('category')

Pandas Apply:   0%|          | 0/56 [00:00<?, ?it/s]

### Feature selection to remove unimportant features

In [10]:
# #Excluding text columns
# predictors = df.drop('classi_fin', axis=1)
# predictors = predictors.drop(predictors.select_dtypes(['object','string']), axis=1)
# response = df['classi_fin']

# X_train, X_val, y_train,y_val = train_test_split(
#     predictors, response, random_state=50, stratify=response
# )

# numerical_cols = predictors.select_dtypes(['int8']).columns.tolist()
# categorical_cols = predictors.select_dtypes(['category']).columns.tolist()

# categorical_imputer = ColumnTransformer(
#     [('cat_imputer', SimpleImputer(strategy='most_frequent'), categorical_cols)],
#     remainder='drop'
# )


# X_train_transformed = pd.DataFrame(
#     categorical_imputer.fit_transform(X_train),
#     columns=categorical_cols,
#     index=X_train.index
# )
# X_train_transformed = pd.concat([X_train_transformed,X_train[numerical_cols]],axis=1)
# X_train_transformed[categorical_cols] = X_train_transformed[categorical_cols].astype('category')

# X_val_transformed = pd.DataFrame(
#     categorical_imputer.transform(X_val),
#     columns=categorical_cols,
#     index=X_val.index
# )
# X_val_transformed = pd.concat([X_val_transformed,X_val[numerical_cols]],axis=1)
# X_val_transformed[categorical_cols] = X_val_transformed[categorical_cols].astype('category')

# categorical_transformer = Pipeline(
#     steps=[
#         ('encoder', CountFrequencyEncoder(encoding_method='frequency'))
#     ]
# )

# numerical_transformer = Pipeline(
#     steps=[
#         ('encoder', MinMaxScaler())
#     ]    
# )

# preprocessor = ColumnTransformer(
#     [
#         ('cat', categorical_transformer, categorical_cols),
#         ('num', numerical_transformer, numerical_cols)
#     ]   
# )

# X_train_transformed = pd.DataFrame(
#     preprocessor.fit_transform(X_train),
#     columns=predictors.columns,
#     index=X_train_transformed.index
# )

# X_val_transformed = pd.DataFrame(
#     preprocessor.transform(X_val),
#     columns=predictors.columns,
#     index=X_val_transformed.index
# )

# encoder = LabelEncoder()
# y_train = encoder.fit_transform(y_train)
# y_val = encoder.transform(y_val)

# classifiers = [
#     RandomForestClassifier(n_jobs=-1, random_state=42, class_weight='balanced_subsample'),
#     XGBClassifier(n_jobs=-1, random_state=42,objective='multi:softax', max_delta_step=1),
#     lgbm.LGBMClassifier(n_jobs=-1,  random_state=42, class_weight='balanced')
# ]

# for classifier in classifiers:
#     pipeline_1 = Pipeline(
#         steps= [
#         ('feature_selection', SelectFromModel(estimator=classifier))
#         ]
#     )
#     pipeline_2 = Pipeline(
#         steps= [
#         ('feature_selection', RFE(estimator=classifier))
#         ]
#     )
    
#     permutation_score = permutation_importance(
#         classifier.fit(X_train_transformed,y_train), X_val_transformed, y_val,
#         random_state=42, scoring='f1_weighted', n_repeats=10
#     )

#     importance = pd.DataFrame(
#         {'features':X_train_transformed.columns, 
#         'f1_weighted':permutation_score['importances_mean']}).sort_values(by='f1_weighted', ascending=False
#     )

#     print(
#         'model: {} \n features selected based on feature importance:{} \n\n'.format(pipeline_1['feature_selection'].estimator,
#         pipeline_1.fit(X_train_transformed,y_train).get_feature_names_out(input_features=None)) 
#         )
#     print(
#         'model: {} \n features_selected based on RFE:{} \n\n'.format(pipeline_2['feature_selection'].estimator,
#         pipeline_2.fit(X_train_transformed,y_train).get_feature_names_out(input_features=None))
#         )
#     print(importance, '\n\n\n')



#Features choosen to continue
feature_selected = [
    'id', 'sem_pri', 'nu_idade_n', 'saturacao', 'antiviral',
    'tp_antivir', 'hospital', 'dose_2_cov', 'dose_ref', 'classi_fin',
    'fnt_in_cov', 'uti','raiox_res', 'dor_abd', 'perd_olft', 'tomo_res',
    'cs_raca', 'cs_zona', 'perd_pala', 'dose_1_cov','vacina_cov'
]

df = df[feature_selected]

# EDA

# Response analysis

In [11]:
df_aux = df['classi_fin'].apply(
    lambda x: 'SARS by influenza' if x == 1
    else 'SARS by other respiratory virus' if x == 2
    else 'SARS by another etiological agent' if x == 3
    else 'unspecified SARS' if x == 4
    else 'SARS by covid-19'
)

# Data preparation and feature selection

In [12]:
#Replace 9 values to mode of each column
df_category = (
    df.select_dtypes(include='category')
    .columns.to_list()
)
column_modes = df[df_category].mode().iloc[0]
for col_name in df_category:
    df[col_name] = df[col_name].replace(9, column_modes[col_name])

In [13]:
X = df.drop(['id','classi_fin'], axis=1)
y = df['classi_fin']

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = pd.Series(y, index=y)

numerical_cols = X.select_dtypes(['int8']).columns.tolist()
categorical_cols = X.select_dtypes(['category']).columns.tolist()
columns_name = categorical_cols + numerical_cols

skf = StratifiedKFold(n_splits=5, shuffle=True)
role = "arn:aws:iam::513734873949:role/FULL_SAGEMAKER"
bucket = 'sagemaker-traintest-respiratory-classification'
prefix1 = 'train'
prefix2 = 'val'
prefixestimat = 'estimator'
sagemaker_session = sagemaker.Session()

hyperparameter_ranges = {
    'eta': sagemaker.tuner.ContinuousParameter(0.1, 0.5),
    'max_depth': sagemaker.tuner.IntegerParameter(2, 9),
    'gamma': sagemaker.tuner.ContinuousParameter(3, 10),
    'min_child_weight': sagemaker.tuner.IntegerParameter(8, 15),
    'subsample': sagemaker.tuner.ContinuousParameter(0.5, 0.7)
}

metrics_list = []

for i, train_index, val_index in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    categorical_transformer = Pipeline(
        steps=[
            ('encoder', CountFrequencyEncoder(encoding_method='frequency'))
        ]
    )

    numerical_transformer = Pipeline(
        steps=[
            ('encoder', MinMaxScaler())
        ]    
    )

    preprocessor = ColumnTransformer(
        [
            ('cat', categorical_transformer, categorical_cols),
            ('num', numerical_transformer, numerical_cols)
        ]   
    )

    X_train = pd.DataFrame(
        preprocessor.fit_transform(X_train),
        columns=columns_name,
        index=X_train.index
    )

    X_val = pd.DataFrame(
        preprocessor.transform(X_val),
        columns=columns_name,
        index=X_val.index
    )
    
    # # Sending to S3
    # train_data = pd.concat([y_train,X_train], axis=1)
    # val_data = pd.concat([y_val,X_val], axis=1)
    # train_data = train_data.rename(columns={0:'classi_fin'})
    # val_data = val_data.rename(columns={0:'classi_fin'})

    # train_data.to_csv('train_processed_fold_{}.csv'.format(i), header=False, index=False)
    # val_data.to_csv('val_processed_fold_{}.csv'.format(i), header=False, index=False)
    

    # train_data_path = sagemaker_session.upload_data(path='train_processed_fold_{}.csv'.format(i)', bucket=bucket, key_prefix=prefix1)
    # val_data_path = sagemaker_session.upload_data(path='val_processed_fold_{}.csv'.format(i)', bucket=bucket, key_prefix=prefix2)

    s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/'.format(bucket, prefix1), content_type='csv')
    s3_input_val = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/'.format(bucket, prefix2), content_type='csv')
    
    container = sagemaker.image_uris.retrieve(region=sagemaker_session.boto_region_name, framework='xgboost', version='1.7-1')


    estimator = sagemaker.estimator.Estimator(
        container,
        role,
        instance_count=1,
        instance_type='ml.m4.xlarge',
        output_path='s3://{}/{}/output_kfold_{}'.format(bucket, prefixestimat, i),
        sagemaker_session=sagemaker_session
    )
    
    tuner = HyperparameterTuner(
    estimator, 
    objective_metric_name='mlogloss',
    hyperparameter_ranges,
    max_jobs=20,
    max_parallel_jobs=3,
    objective_type='Minimize'
)

    tuner.fit({'train': s3_input_train, 'validation': s3_input_val})
    best_estimator = tuner.best_estimator()
    predictor = best_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

    with s3.open(s3_path) as f:
        X_val = pd.read_csv(f)
    
    X_val = preprocessor.transform(X_val)
    
    with s3.open(s3_path) as f:
        y_val = pd.read_csv(f)
    
    
    
    predictions = predictor.predict(data)
    metric = metrics.log_loss(y_val, y_predictions)
    metrics_list.append(metric)

# Machine Learning model selection

## XGBoost

In [ ]:
numerical_cols = X.select_dtypes(['int8']).columns.tolist()
categorical_cols = X.select_dtypes(['category']).columns.tolist()
columns_name = categorical_cols + numerical_cols

categorical_transformer = Pipeline(
    steps=[
        ('encoder', CountFrequencyEncoder(encoding_method='frequency'))
    ]
)

numerical_transformer = Pipeline(
    steps=[
        ('encoder', MinMaxScaler())
    ]    
)

preprocessor = ColumnTransformer(
    [
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ]   
)

clf_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', xgboost.XGBClassifier(random_state=42, objective='multi:softprob'))
    ]
)

param_grid = {
    'classifier__eta': [0.1, 0.5],
    'classifier__max_depth': [2, 9],
    'classifier__min_split_loss': [2,5,10],
    'classifier__min_child_weight': [8, 20],
    'classifier__subsample': [0.5, 0.7],
}

clf_grid = GridSearchCV(clf_pipeline, param_grid=param_grid, scoring="neg_log_loss", cv=2)

outer_cv = cross_validate(clf_grid, X,y, scoring="neg_log_loss", cv=5, n_jobs=-1)

outer_cv_mean_xgboost = outer_cv.test_score.mean()
outer_cv_sd_xgboost = outer_cv.test_score.std()

In [14]:
numerical_cols = X.select_dtypes(['int8']).columns.tolist()
categorical_cols = X.select_dtypes(['category']).columns.tolist()
columns_name = categorical_cols + numerical_cols

categorical_transformer = Pipeline(
    steps=[
        ('encoder', CountFrequencyEncoder(encoding_method='frequency'))
    ]
)

numerical_transformer = Pipeline(
    steps=[
        ('encoder', MinMaxScaler())
    ]    
)

preprocessor = ColumnTransformer(
    [
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ]   
)

X_train = pd.DataFrame(
    preprocessor.fit_transform(X_train),
    columns=columns_name,
    index=X_train.index
)

X_val = pd.DataFrame(
    preprocessor.transform(X_val),
    columns=columns_name,
    index=X_val.index
)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_train = pd.Series(y_train, index=train_index)
y_val = encoder.transform(y_val)
y_val = pd.Series(y_val, index=val_index)

In [15]:
# Create new bucket for processed data
bucket = 'sagemaker-traintest-respiratory-classification'
prefix1 = 'train'
prefix2 = 'test'
prefixestimat = 'estimator'

# # Sending to S3
# train_data = pd.concat([y_train,X_train], axis=1)
# val_data = pd.concat([y_val,X_val], axis=1)
# train_data = train_data.rename(columns={0:'classi_fin'})
# val_data = val_data.rename(columns={0:'classi_fin'})

# train_data.to_csv('train_processed.csv', header=False, index=False)
# val_data.to_csv('validation.csv', header=False, index=False)

In [18]:
sagemaker_session = sagemaker.Session()

# train_data_path = sagemaker_session.upload_data(path='train_processed.csv', bucket=bucket, key_prefix=prefix1)
# val_data_path = sagemaker_session.upload_data(path='validation.csv', bucket=bucket, key_prefix=prefix2)

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/'.format(bucket, prefix1), content_type='csv')
s3_input_val = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/'.format(bucket, prefix2), content_type='csv')

In [22]:
role = "arn:aws:iam::513734873949:role/FULL_SAGEMAKER"

container = sagemaker.image_uris.retrieve(region=sagemaker_session.boto_region_name, framework='xgboost', version='1.7-1')


estimator = sagemaker.estimator.Estimator(container,
                                          role,
                                          instance_count=1,
                                          instance_type='ml.m4.xlarge',
                                          output_path='s3://{}/{}/output'.format(bucket, prefixestimat),
                                          sagemaker_session=sagemaker_session)

estimator.fit({'train': s3_input_train, 'validation': s3_input_val})
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.large')

In [ ]:
# role = "arn:aws:iam::513734873949:role/FULL_SAGEMAKER"

# hyperparameter_ranges = {
#     'eta': ContinuousParameter(0.2, 0.5),
#     'max_depth': IntegerParameter(2, 10),
#     'gamma': ContinuousParameter(2, 10),
#     'min_child_weight': IntegerParameter(8,20),
#     'subsample': "0.7",
#     'objective': "multi:softmax",
    
#     # Add other hyperparameters here
# }

# cv_metric = []
# objective_metric_name = 'validation:mlogloss'

# container = sagemaker.image_uris.retrieve(region=sagemaker_session.boto_region_name, framework='xgboost', version='1.7-1')


# estimator = sagemaker.estimator.Estimator(container,
#                                           role,
#                                           instance_count=1,
#                                           instance_type='ml.m4.xlarge',
#                                           output_path='s3://{}/{}/output'.format(bucket, prefixestimat),
#                                           sagemaker_session=sagemaker_session)



# tuner = HyperparameterTuner(
#     estimator,
#     objective_metric_name,
#     hyperparameter_ranges,
#     max_jobs=20,
#     max_parallel_jobs=3,
#     objective_type='Minimize'
# )
# tuner.fit({'train': s3_input_train, 'validation': s3_input_val})
# best_estimator = tuner.best_estimator()
# predictor = best_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
# predictions = predictor.predict(s3_input_val)
# metric_result = metrics.log_loss(y_true, predictions)
# cv_metric.append(metric_result)
# predictor.delete_endpoint()